In [ ]:
# Add the root directory to the path to allow importing the module
import sys
sys.path.append('/mn/stornext/u3/avijeetp/codes/ISPy')
sys.path.append('/mn/stornext/u3/avijeetp/codes/pyMilne')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import MilneEddington as ME
import crisp
import imtools as im
import time
from astropy.io import fits
from ISPy.io import solarnet
from einops import rearrange
print("All modules loaded")

In [ ]:

def loadFits(name, tt=0):
    datafits = fits.open(name, 'readonly')[0].data[tt,...]
    # Fill nans with 0s:
    datafits = np.nan_to_num(datafits)
    # Normalize the data to average:
    qs_nom = np.nanmean(datafits[0,0,:,:])
    datafits = rearrange(datafits, 'ns nw ny nx -> ny nx ns nw')/qs_nom
    return np.ascontiguousarray(datafits, dtype='float64')


In [ ]:
def loadCmap(name, tt=0):
    """
    Load the 'WCSDVARR' key from a FITS file for a specific time index and fill NaNs with 0s.

    Parameters:
    name (str): The name of the FITS file.
    tt (int): The time index.

    Returns:
    np.ndarray: The dlambda array for the specified time index.
    """
    with fits.open(name, 'readonly') as hdulist:
        # Extract the data for the specific time index tt
        dlambda = hdulist['WCSDVARR'].data[tt,0,0,:,:]

    # Fill NaNs with 0s
    dlambda = np.nan_to_num(dlambda)

    return np.ascontiguousarray(dlambda, dtype='float64')

In [ ]:
def getWavelengths(name):
    wav_array = solarnet.get_wav(name) * 10 # convert from nm to Angstrom
    return np.ascontiguousarray(wav_array, dtype='float64')

In [ ]:
def findgrid(w, dw, extra=5):
    """
    Findgrid creates a regular wavelength grid 
    with a step of dw that includes all points in 
    input array w. It adds extra points at the edges
    for convolution purposes

    Returns the new array and the positions of the
    wavelengths points from w in the new array
    """
    nw = np.int32(np.rint(w/dw))
    nnw = nw[-1] - nw[0] + 1 + 2*extra

    iw = np.arange(nnw, dtype='float64')*dw - extra*dw + w[0]

    idx = np.arange(w.size, dtype='int32')
    for ii in range(w.size):
        idx[ii] = np.argmin(np.abs(iw-w[ii]))

    return iw, idx


In [ ]:

class container:
    def __init__(self):
        pass

In [ ]:
#
# Decide to work in float32 or float64
#
dtype = 'float32'
nthreads = 80

In [ ]:
# datadir = '/mn/stornext/d18/lapalma/reduc/2020/2020-08-07/CRISP/cubes_nb/'
# crisp_im_file = 'nb_6173_2020-08-07T08:22:14_scans=0-56_stokes_corrected_im.fits'
datadir = '/mn/stornext/d18/lapalma/reduc/2024/2024-05-21/CRISP/cubes_nb/'
crisp_im_file = 'nb_6173_2024-05-21T10:19:04_10:19:04=0-52_stokes_corrected_im.fits'
crisp_im = datadir + crisp_im_file

In [ ]:

#
# Load data, wavelength array and cmap
#
l = container()
container.iwav = getWavelengths(crisp_im)
container.d = loadFits(crisp_im, tt=0) 
container.cmap = loadCmap(crisp_im, tt=0)

In [ ]:
l.iwav

# Minimum step:
dw = np.min(np.diff(l.iwav))
# dw = round((lambda*10. - lc) * 1000.) ; offset in mA
dw = round(dw*1000.)/1000. # avoid floating point errors

dw


In [ ]:

# The inversions need to account for the instrumental
# profile, which involve convolutions. The convolutions
# must be done in a wavelength grid that is at least
# 1/2 of the FWHM of the instrumental profile. In the
# case of CRISP that would be ~55 mA / 2 = ~27.5 mA
#
# Get finer grid for convolutions purposes
# Since we only observed at the lines, let's create
# two regions, one for each line
#
# The observed line positions are not equidistant, the
# Fe I 6301 points only fit into a regular grid of 5 mA
# whereas the Fe I 6302 can fit into a 15 mA grid
#
iw, idx = findgrid(l.iwav, dw)  # Fe I 6173

print(iw,  idx)

In [ ]:
l.d.shape

In [ ]:

#
# Now we need to create a data cube with the fine grid
# dimensions. All observed points will contribute to the
# inversion. The non-observed ones will have zero weight
# but will be used internally to properly perform the
# convolution of the synthetic spectra
#


ny, nx = l.d.shape[0:2]
obs = np.zeros((ny, nx, 4, iw.size), dtype=dtype, order='c')

for ss in range(4):
    for ii in range(idx.size):
        obs[:, :, ss, idx[ii]] = l.d[:, :, ss, ii]


In [ ]:

#
# Create sigma array with the estimate of the noise for
# each Stokes parameter at all wavelengths. The extra
# non-observed points will have a very large noise (1.e34)
# (zero weight) compared to the observed ones (3.e-3)
# Since the amplitudes of Stokes Q,U and V are very small
# they have a low imprint in Chi2. We can artificially
# give them more weight by lowering the noise estimate.
#
sig = np.zeros((4, iw.size), dtype=dtype) + 1.e32
sig[:, idx] = 5.e-3
sig[1:2, idx] /= 9.0
sig[3, idx] /= 4.0



In [ ]:

#
# Init Me class. We need to create two regions with the
# wavelength arrays defined above and a instrumental profile
# for each region in with the same wavelength step
#
tw = (np.arange(iw.size, dtype=dtype)-iw.size//2)*dw


In [ ]:
# Central wavelength of the line:
l0 = iw[iw.size//2]
tr = crisp.crisp(l0).dual_fpi(tw, erh=-0.001)

regions = [[iw, tr/tr.sum()]]
lines = [int(l0)]
me = ME.MilneEddington(regions, lines, nthreads=nthreads, precision=dtype)


In [ ]:

#
# Init model parameters
#
iPar = np.float64([1500, 2.2, 1.0, -0.5, 0.035, 50., 0.1, 0.24, 0.7]) # [B_tot, theta_B, chi_B, gamma_B, v_los, eta_0, Doppler width, damping, s0, s1]
Imodel = me.repeat_model(iPar, ny, nx)


In [ ]:
#
# Run a first cycle with 4 inversions of each pixel (1 + 3 randomizations) of simple pixel-wise inversion
#
t0 = time.time()
Imodel, syn, chi2 = me.invert(
    Imodel, obs, sig, nRandom=4, nIter=25, chi2_thres=1.0, mu=0.54184232)
t1 = time.time()
print("dT = {0}s -> <Chi2> = {1}".format(t1-t0, chi2.mean()))

In [ ]:
def plot_output(mos):
    f, ax = plt.subplots(nrows=3, ncols=3, figsize=(30, 30))
    ax1 = ax.flatten()

    cmaps = ['gist_gray', 'RdGy', 'RdGy', 'bwr', 'gist_gray', 'gist_gray',
                'gist_gray', 'gist_gray', 'gist_gray']
    labels = ['B [G]', 'inc [rad]', 'azi [rad]', 'Vlos [km/s]', 
                'vDop [Angstroms]', 'lineop', 'damp', 'S0', 'S1']

    extent = np.float32((0, nx, 0, ny))*0.059
    for ii in range(9):
        if (ii != 3):
            a = ax1[ii].imshow(im.histo_opt(mos[:, :, ii]), cmap=cmaps[ii],
                                interpolation='nearest', extent=extent, aspect='equal')
        else:
            a = ax1[ii].imshow(mos[:, :, ii], cmap=cmaps[ii], interpolation='nearest',
                                extent=extent, vmax=4, vmin=-4, aspect='equal')
        f.colorbar(a, ax=ax1[ii], orientation='vertical', label=labels[ii])

    for jj in range(3):
        for ii in range(3):
            if (jj != 2):
                ax[jj, ii].set_xticklabels([])
            if (ii != 0):
                ax[jj, ii].set_yticklabels([])

    f.set_tight_layout(True)
    print("saving figure with results -> fig_results.pdf")
    f.savefig('fig_results.pdf', dpi=250, format='pdf')
    f.show()


    # Create a new figure with two subplots
    f2, ax2 = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))

    # Blos map:
    Blos = mos[:, :, 0] * np.cos(mos[:, :, 1])
    im1 = ax2[0].imshow(np.rot90(Blos.T), cmap='Greys_r', interpolation='nearest', aspect='equal', vmin=-1500, vmax=1500)
    plt.colorbar(im1, ax=ax2[0])

    # Bhor map:
    Bhor = mos[:, :, 0] * np.sin(mos[:, :, 1])
    im2 = ax2[1].imshow(np.rot90(Bhor.T), cmap='Greys_r', interpolation='nearest', aspect='equal', vmin=-1500, vmax=1500)
    plt.colorbar(im2, ax=ax2[1])

    # Display the figure
    f2.show()
    

In [ ]:
plot_output(np.squeeze(Imodel))

# nRandom = 4:
# 1) inversion from Imodel during nIter iterations
# 2 and later=> inversion from Imodel + np.random(0.,x) using nIter iterations



In [ ]:
#
# Run second cycle, starting from the smoothed guessed model
#
t0 = time.time()
Imodel, syn, chi2 = me.invert(
    Imodel, obs, sig, nRandom=1, nIter=25, chi2_thres=1.0, mu=0.54184232)
t1 = time.time()
print("dT = {0}s -> <Chi2> = {1}".format(t1-t0, chi2.mean()))

In [ ]:
plot_output(np.squeeze(Imodel))

In [ ]:

#
# Run a first cycle with 4 inversions of each pixel (1 + 3 randomizations)
#
t0 = time.time()
mo, syn, chi2 = me.invert_spatially_regularized(Imodel, obs, sig,  nIter=25, chi2_thres=1.0, mu=0.54184232, alpha=30., alphas=np.float32([
                                                2, 0.5, 2, 0.01, 0.1, 0.01, 0.1, 0.01, 0.01]), method=1, delay_bracket=3)
t1 = time.time()
print("dT = {0}s -> <Chi2> (including regularization) = {1}".format(t1-t0, chi2))


In [ ]:

mos = np.squeeze(mo)
plot_output(mos)

In [ ]:
l.cmap.shape

In [ ]:

#
# Correct velocities for cavity error map from CRISP
#
mos = np.squeeze(mo) # Remove the singleton dimension in the model and make the shape (ny, nx, 9) from (1, ny, nx, 9)
v_cmap_corrected = mos[:,:,3] + (l.cmap * 10) / l0 * 2.9e5
# mos[:,:,3] += l.cmap+0.45 # The 0.45 is a global offset that seems to make the umbra at rest


In [ ]:
# imshow mos[:,:,3] and v_cmap_corrected to see the difference
fig = plt.figure(figsize=(20, 10))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
im = ax1.imshow(mos[:,:,3], cmap='seismic', interpolation='nearest', aspect='equal')
im2 = ax2.imshow(v_cmap_corrected, cmap='seismic', interpolation='nearest', aspect='equal')
# add colorbars
plt.colorbar(im, ax=ax1)
plt.colorbar(im2, ax=ax2)

plt.show()


In [ ]:
mos[:,:,3].shape

In [ ]:
l.cmap.shape

In [ ]:
mos[:,:,3][500,500]

In [ ]:
l.cmap[500,500]

In [ ]:

#
# make plots
#
# plt.ion()
f, ax = plt.subplots(nrows=3, ncols=3, figsize=(30, 30))
ax1 = ax.flatten()

cmaps = ['gist_gray', 'RdGy', 'RdGy', 'bwr', 'gist_gray', 'gist_gray',
            'gist_gray', 'gist_gray', 'gist_gray']
labels = ['B [G]', 'inc [rad]', 'azi [rad]', 'Vlos [km/s]', 
            'vDop [Angstroms]', 'lineop', 'damp', 'S0', 'S1']

extent = np.float32((0, nx, 0, ny))*0.059
for ii in range(9):
    if (ii != 3):
        a = ax1[ii].imshow(im.histo_opt(mos[:, :, ii]), cmap=cmaps[ii],
                            interpolation='nearest', extent=extent, aspect='equal')
    else:
        a = ax1[ii].imshow(mos[:, :, ii], cmap=cmaps[ii], interpolation='nearest',
                            extent=extent, vmax=4, vmin=-4, aspect='equal')
    f.colorbar(a, ax=ax1[ii], orientation='vertical', label=labels[ii])

for jj in range(3):
    for ii in range(3):
        if (jj != 2):
            ax[jj, ii].set_xticklabels([])
        if (ii != 0):
            ax[jj, ii].set_yticklabels([])

f.set_tight_layout(True)
print("saving figure with results -> fig_results.pdf")
f.savefig('fig_results.pdf', dpi=250, format='pdf')
f.show()
